# Quantitative analysis of EELS finestructure

In [1]:
%matplotlib qt4

In [2]:
import matplotlib as mpl
mpl.rcParams['image.cmap'] = 'viridis'

In [3]:
import hyperspy.api as hs
import numpy as np

 Check that the Hartree-Slater determinants are loaded:
 EELS -> Model -> GOS directory. Then go to the  UNZIPPED FOLDER.

In [ ]:
hs.preferences.gui()

In [66]:
s = hs.load("EELS Spectrum Image (high-loss) (aligned).dm3")
s_ll = hs.load("EELS Spectrum Image (low-loss) (aligned).dm3")

In [67]:
#s.plot()

In [68]:
#s_ll.plot()

Edge energy parameters. [Start background , end background , start edge, end edge]. Check this!

In [69]:
s_o_edge_E  = [503.,527.,517.,600.]
s_mn_edge_E = [602.,632.,620.,702.]

### Processing the data

We fix the energy offset by using the `align_zero_loss_peak` method (PE already did this):

In [70]:
s_ll.align_zero_loss_peak(subpixel=True,calibrate=True,also_align=[s])


Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	-0.080
std:	0.000

min:	-0.080
Q1:	-0.080
median:	-0.080
Q3:	-0.080
max:	-0.080


In [71]:
s.change_dtype(float)
s_ll.change_dtype(float)

Now we can extract the different EELS edges, to make it easier to visualize them. For this we use the `isig` method, which is used to slice the signal axes. (The `inav` method is used to slice in navigation axes.)

In [72]:
s_o = s.isig[s_o_edge_E[0]:s_o_edge_E[3]].deepcopy()
s_mn = s.isig[s_mn_edge_E[0]:s_mn_edge_E[3]].deepcopy()

Then remove the power law background using the `remove_background` method. The fitting range for the background can either by specified using a GUI, or the `signal_range` variable. `fast=False` is used to get a better fit, but might take longer time for large datasets.

In [73]:
s_o = s_o.remove_background(signal_range=(s_o_edge_E[0], s_o_edge_E[1]), fast = True)

In [74]:
s_mn = s_mn.remove_background(signal_range=(s_mn_edge_E[0], s_mn_edge_E[1]), fast = True)

Add the Hartree-Slater cross section for Manganese and Oxygen to the signal, so that the models can be created next.

In [75]:
s_mn.add_elements(('Mn',))

In [76]:
s_o.add_elements(('O',))

## Modelling the Mn-L$_{2,3}$ core loss edge

Firstly, we need to make a model object from the core loss and low loss signals. The low loss is important for this, since it allows for the model to be convolved with low loss. This makes the fitting better, and automatically takes into account the effect of multiple scattering. The low loss is added by using the `ll` variable, and since we've already removed the power law background we use the `auto_background=False`.

The low loss signal and the core loss signal must have the same navigation dimensions.

In [78]:
s_mn.metadata.Signal.binned = False
s_ll.metadata.Signal.binned = False

m_mn = s_mn.create_model(ll=s_ll, auto_background=False)


HyperSpy models work by containing one or several components. These components are then fitted to the experimental signal.

Currently, the model contains two components for modelling the Mn-L$_{2,3}$ EELS edge, which can be accessed in:

In [79]:
m_mn.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Mn_L3 |               Mn_L3 |          EELSCLEdge
   1 |               Mn_L2 |               Mn_L2 |          EELSCLEdge

We run the fitting process to get some reasonable initial positions

In [80]:
m_mn.multifit()

The `onset_energy` parameter can be changed interactively using `enable_adjust_position`. Move the onset to a little bit before the L$_3$ peak onset. (This can also be done by doing `m_mn.components.Mn_L3.onset_energy.value = 638`)

In [81]:
#m_mn.enable_adjust_position()

In [82]:
m_mn.components.Mn_L3.onset_energy.value = 638

This only sets the value temporarily for this one specific position. The `assign_current_value_to_all` method is used to make it permanent for all the positions

In [83]:
m_mn.components.Mn_L3.onset_energy.assign_current_value_to_all()

The `onset_energy` parameter can be changed interactively using `enable_adjust_position`. Move the onset to a little bit before the L$_2$ peak onset. (This can also be done by doing `m_mn.components.Mn_L2.onset_energy.value = 649.0`)

In [84]:
#m_mn.components.Mn_L2.onset_energy.value = 649

In [85]:
m_mn.components.Mn_L2.onset_energy.assign_current_value_to_all()

The next step is making the fitting function ignore the intense L$_3$ and L$_2$ peaks, which we'll fit later using Gaussians. The regions used for the fitting can be specified interactively using `m_mn.set_signal_range()`. Set the signal range between 603 and 636 eV.

In [87]:
m_mn.set_signal_range(603, 636)

The use `add_signal_range` to add a second signal range at the lowest point in the edge background (659 to 693 eV)

In [88]:
m_mn.add_signal_range(659,693)

Now, the fitting algorithm will only fit the model to the regins we just specified

In [89]:
m_mn.multifit()

First we reset the signal range, so we can see the whole model.

In [90]:
m_mn.reset_the_signal_range()

Now the the edge background looks much better

In [91]:
m_mn.plot()

We can now lock the EELSCLEdge component

In [92]:
m_mn.components.Mn_L2.set_parameters_not_free()
m_mn.components.Mn_L3.set_parameters_not_free()

This means the component will not change when fitting

### Adding the Gaussians

First we need to make two Gaussian components

In [93]:
g1 = hs.model.components1D.Gaussian()
g2 = hs.model.components1D.Gaussian()

These are added to the model using `append` (just like a standard Python list)

In [94]:
m_mn.append(g1)
m_mn.append(g2)

In [95]:
m_mn.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |               Mn_L3 |               Mn_L3 |          EELSCLEdge
   1 |               Mn_L2 |               Mn_L2 |          EELSCLEdge
   2 |            Gaussian |            Gaussian |            Gaussian
   3 |          Gaussian_0 |          Gaussian_0 |            Gaussian

Before trying to fit them to the signal, we should give them some reasonable initial values. This can be done manually, or more easily using the `fit_component` method. It works by locking all other components, and only fitting the specified component across the range which is selected.

Drag the span across the L$_3$ (leftmost peak, 637.45 to 648.32 eV), untick `only_current`, and press fit.

In [96]:
m_mn.fit_component(g1, signal_range=(637.45, 648.32), only_current = False)

Repeat this for the g2 component, but now for the L$_2$ peak. Drag the span from 649 to 657 eV, untick only_current, then hit fit.

In [97]:
m_mn.fit_component(g2,signal_range=(649, 657), only_current = False)

Now that the two Gaussian components have some reasonable initial values, we can try fitting both of them at the same time. But first we want to change the signal range so it does not contain all of the edge background. Use `set_signal_range` to set the fitting range from 635 to 661 eV.

In [98]:
m_mn.set_signal_range(635,661)

The fitting is better, but not perfect. One issue is the intensity in the edge background "behind" the two main peaks is not very good at the moment. This is due to the plural scattering from the Gaussians now contributing intensity.

This is solved by letting the intensity parameter for the EELSCLEdge component be free.

In [99]:
m_mn.components.Mn_L3.intensity.free = True

In [100]:
m_mn.multifit()

In [101]:
m_mn.reset_signal_range()

Now the EELSCLEdge fits better

In [112]:
m_mn[0].fine_structure_width = 80
m_mn[1].fine_structure_width = 80

a = np.absolute(m_mn[2].A.as_signal())
b = np.absolute(m_mn[3].A.as_signal())

sigma_L3_integrated = np.absolute(m_mn[0].get_fine_structure_as_signal1D().sum(-1)).transpose()
sigma_L2_integrated = np.absolute(m_mn[1].get_fine_structure_as_signal1D().sum(-1)).transpose()

lambda_2 = np.absolute(m_mn[0].intensity.as_signal())

s_L32_sum = (a+b)/(lambda_2*(sigma_L2_integrated+sigma_L3_integrated))
s_L32_sum.metadata.General.title = 'I_L3 + I_L2'

s_L32_sum_av2 = []
s_L32_sum_s2 = []

s_L32_sum_av2.append(s_L32_sum.isig[0.300:0.600].mean(0))
s_L32_sum_s2.append(s_L32_sum.isig[0.300:0.600].std(0)/np.sqrt(len(s_L32_sum.isig[0.300:0.600].data)))

s_L32_sum_av2.append(s_L32_sum.isig[0.600:0.900].mean(0))
s_L32_sum_s2.append(s_L32_sum.isig[0.600:0.900].std(0)/np.sqrt(len(s_L32_sum.isig[0.600:0.900].data)))

s_L32_sum_av2.append(s_L32_sum.isig[0.0:0.300].mean(0))
s_L32_sum_s2.append(s_L32_sum.isig[0.0:0.300].std(0)/np.sqrt(len(s_L32_sum.isig[0.0:0.300].data)))

means2 = [s_L32_sum_av2[0].data[0], s_L32_sum_av2[1].data[0], s_L32_sum_av2[2].data[0]]
errors2 = [s_L32_sum_s2[0].data[0], s_L32_sum_s2[1].data[0], s_L32_sum_s2[2].data[0]]


mpl.pyplot.errorbar([0.450], means2[0], xerr=0.150, yerr=0, fmt='o', 
                    ecolor='b', mfc = 'g', capthick=2,capsize=10, elinewidth=5)

mpl.pyplot.errorbar([0.450], means2[0], xerr=0, yerr=errors2[0], fmt='o', 
                    ecolor='r', mfc = 'g', capthick=2,capsize=10, elinewidth=5)


mpl.pyplot.errorbar([0.750], means2[1], xerr=0.150, yerr=0, fmt='o', 
                    ecolor='b', mfc = 'g', capthick=2,capsize=10, elinewidth=5)

mpl.pyplot.errorbar([0.750], means2[1], xerr=0, yerr=errors2[1], fmt='o', 
                    ecolor='r', mfc = 'g', capthick=2,capsize=10, elinewidth=5)

mpl.pyplot.errorbar([0.150], means2[2], xerr=0.150, yerr=0, fmt='o', 
                    ecolor='b', mfc = 'g', capthick=2,capsize=10, elinewidth=5)

mpl.pyplot.errorbar([0.150], means2[2], xerr=0, yerr=errors2[0], fmt='o', 
                    ecolor='r', mfc = 'g', capthick=2,capsize=10, elinewidth=5)


mpl.pyplot.axvline(x=0.3, linewidth=4, color='g', linestyle='dashed')
mpl.pyplot.axvline(x=0.6, linewidth=4, color='g', linestyle='dashed')

mpl.pyplot.xticks([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
mpl.pyplot.title('Gaussian fit, $L_3+L_2$ Normalized Amplitude, Averaged with SE')
mpl.pyplot.xlabel('$\mu$m')
mpl.pyplot.ylabel('$L_3+L_2$')
mpl.rcParams.update({'font.size': 30})
mpl.pyplot.xlim([0, 0.9])

(0, 0.9)

In [111]:
a = np.absolute(m_mn[2].A.as_signal())
b = np.absolute(m_mn[3].A.as_signal())

s_L32_ratio = np.absolute(a)/np.absolute(b)
s_L32_ratio.metadata.General.title = 'I_L3/I_L2'

s_L32_ratio_av2 = []
s_L32_ratio_s2 = []


s_L32_ratio_av2.append(s_L32_ratio.isig[0.450:0.650].mean(0))
s_L32_ratio_s2.append(s_L32_ratio.isig[0.450:0.650].std(0)/np.sqrt(len(s_L32_ratio.isig[0.450:0.650].data)))

s_L32_ratio_av2.append(s_L32_ratio.isig[0.640:0.940].mean(0))
s_L32_ratio_s2.append(s_L32_ratio.isig[0.640:0.940].std(0)/np.sqrt(len(s_L32_ratio.isig[0.640:0.940].data)))

s_L32_ratio_av2.append(s_L32_ratio.isig[0.0:0.300].mean(0))
s_L32_ratio_s2.append(s_L32_ratio.isig[0.0:0.300].std(0)/np.sqrt(len(s_L32_ratio.isig[0.0:0.300].data)))

means2 = [s_L32_ratio_av2[0].data[0], s_L32_ratio_av2[1].data[0], s_L32_ratio_av2[2].data[0]]
errors2 = [s_L32_ratio_s2[0].data[0], s_L32_ratio_s2[1].data[0], s_L32_ratio_s2[2].data[0]]


mpl.pyplot.errorbar([0.550], means2[0], xerr=0.100, yerr=0, fmt='o', 
                    ecolor='b', mfc = 'g', capthick=2,capsize=10, elinewidth=5)

mpl.pyplot.errorbar([0.550], means2[0], xerr=0, yerr=errors2[0], fmt='o', 
                    ecolor='r', mfc = 'g', capthick=2,capsize=10, elinewidth=5)

mpl.pyplot.errorbar([0.790], means2[1], xerr=0.150, yerr=0, fmt='o', 
                    ecolor='b', mfc = 'g', capthick=2,capsize=10, elinewidth=5)

mpl.pyplot.errorbar([0.790], means2[1], xerr=0, yerr=errors2[1], fmt='o', 
                    ecolor='r', mfc = 'g', capthick=2,capsize=10, elinewidth=5)

mpl.pyplot.errorbar([0.150], means2[2], xerr=0.150, yerr=0, fmt='o', 
                    ecolor='b', mfc = 'g', capthick=2,capsize=10, elinewidth=5)

mpl.pyplot.errorbar([0.150], means2[2], xerr=0, yerr=errors2[0], fmt='o', 
                    ecolor='r', mfc = 'g', capthick=2,capsize=10, elinewidth=5)


mpl.pyplot.xticks([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
mpl.pyplot.title('Gaussian fit, $L_3/L_2$ Amplitude Ratio, Averaged with SE')
mpl.pyplot.xlabel('$\mu$m')
mpl.pyplot.ylabel('$L_3/L_2$')
mpl.rcParams.update({'font.size': 30})
mpl.pyplot.xlim([0, 0.942])

(0, 0.942)

In [104]:
m_mn[0].get_fine_structure_as_signal1D().plot()

In [105]:
m_mn.plot()

In [106]:
m_mn.print_current_values()

Components	Parameter	Value
Mn_L3
		intensity	6.8657e-05
Mn_L2
Gaussian
		A	0.152032
		centre	641.778
		sigma	1.43289
Gaussian_0
		A	0.0652113
		centre	652.631
		sigma	1.46371


In [107]:
s_mn.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EELS
│       │       ├── aperture_size = 5.0
│       │       ├── collection_angle = 66.9000015258789
│       │       ├── frame_number = 1
│       │       └── spectrometer = GIF Quantum ER
│       ├── acquisition_mode = STEM
│       ├── beam_current = 0.0
│       ├── beam_energy = 200.0
│       ├── camera_length = 15.0
│       ├── convergence_angle = 27.0
│       ├── magnification = 200000.0
│       └── microscope = JEM-ARM200F
├── General
│   ├── original_filename = EELS Spectrum Image (high-loss) (aligned).dm3
│   └── title = EELS Spectrum Image (high-loss) (aligned)
├── Sample
│   ├── description = DWNC
│   └── elements = ['Mn']
└── Signal
    ├── binned = False
    ├── quantity = Intensity (Counts)
    └── signal_type = EELS

If I do a test, I get the correct amplitude in my model. Hence, it is very strange that it gives the wrong amplitude above.

In [108]:
g1 = hs.model.components1D.Gaussian(A=25.0, sigma=4.0, centre=2.0)
x = np.arange(-20, 20, 0.01)
data = np.zeros((5,5,4000))
data[:] = g1.function(x).reshape((1,1,4000))
s = hs.signals.Signal1D(data)

s.axes_manager._axes[-1].offset = -20
s.axes_manager._axes[-1].scale = 0.01
#s.plot()

g2 = hs.model.components1D.Gaussian()
m = s.create_model()
m.append(g2)
m.fit_component(g2)
m.multifit()

In [109]:
m.print_current_values()

Components	Parameter	Value
Gaussian
		A	25
		centre	2
		sigma	4
